# Ibis v6.1.0

02 August 2023

## Overview

Ibis 6.1.0 is a minor release that includes new features, backend improvements, bug fixes, documentation improvements, and refactors. We are excited to see further adoption of the dataframe interchange protocol enabling visualization and other libraries to be used more easily with Ibis.

You can view the full changelog in [the release notes](/release_notes/).

If you're new to Ibis, see [how to install](/install/) and [the getting started tutorial](/tutorial/getting_started/).

To follow along with this blog, ensure you're on `'ibis-framework>=6.1,<7'`. First, we'll setup Ibis and fetch some sample data to use.

In [1]:
import ibis
import ibis.selectors as s

ibis.__version__

'6.1.0'

In [2]:
# interactive mode for demo purposes
ibis.options.interactive = True

In [3]:
t = ibis.examples.penguins.fetch()
t = t.mutate(year=t["year"].cast("str"))
t.limit(3)

┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃ species ┃ island    ┃ bill_length_mm ┃ bill_depth_mm ┃ flipper_length_mm ┃ body_mass_g ┃ sex    ┃ year   ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│ string  │ string    │ float64        │ float64       │ int64             │ int64       │ string │ string │
├─────────┼───────────┼────────────────┼───────────────┼───────────────────┼─────────────┼────────┼────────┤
│ Adelie  │ Torgersen │           39.1 │          18.7 │               181 │        3750 │ male   │ 2007   │
│ Adelie  │ Torgersen │           39.5 │          17.4 │               186 │        3800 │ female │ 2007   │
│ Adelie  │ Torgersen │           40.3 │          18.0 │               195 │        3250 │ female │ 2007   │
└─────────┴───────────┴────────────────┴───────────────┴───────────────────┴─────────────┴────────┴────────┘

## Ecosystem integrations

With the introduction of `__dataframe__` support in v6.0.0 and efficiency improvements in this release, Ibis now works with [Altair](https://altair-viz.github.io/index.html), [Plotly](https://plotly.com/python/), [plotnine](https://plotnine.readthedocs.io/en/stable/), and any other visualization library that implements the protocol. This enables passing Ibis tables directly to visualization libraries without a `.to_pandas()` or `to_pyarrow()` call for any of the 15+ backends supported, with data efficiently transferred through Apache Arrow.

**Important**: as of writing this blog, a developer version of Altair is required.

In [4]:
import altair as alt

chart = (
    alt.Chart(t, width=500)
    .mark_point(size=30)
    .encode(
        x=alt.X("bill_length_mm").scale(zero=False),
        y=alt.Y("bill_depth_mm").scale(zero=False),
        color="species",
        shape="island",
    )
    .interactive()
)

chart

alt.Chart(...)

A more modular, composable, and scalable way of working with data is taking shape with `__dataframe__` and `__array__` support in Ibis and increasingly the Python data ecosystem. Let's combine the above with PCA after some preprocessing in Ibis to visualize all numeric columns in 2D.

In [5]:
def transform(t):
    # compute the z score
    t = t.mutate(
        s.across(s.numeric(), {"zscore": lambda x: (x - x.mean()) / x.std()})
    ).dropna()  # drop rows with missing values
    return t


f = transform(t)
f

┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ species ┃ island    ┃ bill_length_mm ┃ bill_depth_mm ┃ flipper_length_mm ┃ body_mass_g ┃ sex    ┃ year   ┃ bill_length_mm_zscore ┃ bill_depth_mm_zscore ┃ flipper_length_mm_zscore ┃ body_mass_g_zscore ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ string  │ string    │ float64        │ float64       │ int64             │ int64       │ string │ string │ float64               │ float64              │ float64                  │ float64            │
├─────────┼───────────┼────────────────┼───────────────┼───────────────────┼─────────────┼────────┼────────┼───────────────────────┼──────────────────────┼──────────────────────────┼────────────────────┤
│ Adelie  │ Torgersen │           39.1 │          18.7 │               181 │        3750 │ male   │ 2007   │             -0.883205 │             0.784300 │                -1.416272 │          -0.563317 │
│ Adelie  │ Torgersen │           39.5 │          17.4 │               186 │        3800 │ female │ 2007   │             -0.809939 │             0.126003 │                -1.060696 │          -0.500969 │
│ Adelie  │ Torgersen │           40.3 │          18.0 │               195 │        3250 │ female │ 2007   │             -0.663408 │             0.429833 │                -0.420660 │          -1.186793 │
│ Adelie  │ Torgersen │           36.7 │          19.3 │               193 │        3450 │ female │ 2007   │             -1.322799 │             1.088129 │                -0.562890 │          -0.937403 │
│ Adelie  │ Torgersen │           39.3 │          20.6 │               190 │        3650 │ male   │ 2007   │             -0.846572 │             1.746426 │                -0.776236 │          -0.688012 │
│ Adelie  │ Torgersen │           38.9 │          17.8 │               181 │        3625 │ female │ 2007   │             -0.919837 │             0.328556 │                -1.416272 │          -0.719186 │
│ Adelie  │ Torgersen │           39.2 │          19.6 │               195 │        4675 │ male   │ 2007   │             -0.864888 │             1.240044 │                -0.420660 │           0.590115 │
│ Adelie  │ Torgersen │           41.1 │          17.6 │               182 │        3200 │ female │ 2007   │             -0.516876 │             0.227280 │                -1.345156 │          -1.249141 │
│ Adelie  │ Torgersen │           38.6 │          21.2 │               191 │        3800 │ male   │ 2007   │             -0.974787 │             2.050255 │                -0.705121 │          -0.500969 │
│ Adelie  │ Torgersen │           34.6 │          21.1 │               198 │        4400 │ male   │ 2007   │             -1.707443 │             1.999617 │                -0.207315 │           0.247203 │
│ …       │ …         │              … │             … │                 … │           … │ …      │ …      │                     … │                    … │                        … │                  … │
└─────────┴───────────┴────────────────┴───────────────┴───────────────────┴─────────────┴────────┴────────┴───────────────────────┴──────────────────────┴──────────────────────────┴────────────────────┘

In [6]:
from sklearn.decomposition import PCA

# select "features" as X
X = f.select(s.contains("zscore"))

# get the the first 2 principal components to visualize
n_components = 2
pca = PCA(n_components=n_components).fit(X)

# transform the table to get the principal components
t_pca = ibis.memtable(pca.transform(X)).relabel({"col0": "pc1", "col1": "pc2"})

# join the original table with the PCA table, assuming the order is the same
f = f.mutate(row_number=ibis.row_number().over()).join(
    t_pca.mutate(row_number=ibis.row_number().over()), "row_number"
)

# plot the first 2 principal components
chart = (
    alt.Chart(f, width=500)
    .mark_point(size=30)
    .encode(
        x=alt.X("pc1").scale(zero=False),
        y=alt.Y("pc2").scale(zero=False),
        color="species",
        shape="island",
    )
    .interactive()
)
chart

alt.Chart(...)

## Backends

Numerous backends received improvements. See the [release notes](/release_notes/) for more details.

### DataFusion

The DataFusion backend (and a few others) received several improvements from community member [@mesejo](https://github.com/mesejo) with memtables and many new operations now supported. Some highlights include:

In [7]:
url = ibis.literal("https://ibis-project.org/concepts/why_ibis")
con = ibis.datafusion.connect()

con.execute(url.host())

'ibis-project.org'

In [8]:
con.execute(url.path())

'/concepts/why_ibis'

In [9]:
con.execute(ibis.literal("aaabbbaaa").re_search("bbb"))

True

In [10]:
con.execute(ibis.literal(5.56).ln())

1.715598108262491

In [11]:
con.execute(ibis.literal(5.56).log10())

0.7450747915820575

In [12]:
con.execute(ibis.literal(5.56).radians())

0.09704030641088471

### BigQuery

Some remaining gaps in `CREATE TABLE` DDL options for BigQuery have been filled in, including the ability to pass in `overwrite=True` for table creation.


### PySpark

The PySpark backend now supports reading/writing Delta Lake tables. Your PySpark session must be configured to use the Delta Lake package and you must have the `delta` package installed in your environment.


```python
t = ibis.read_delta("/path/to/delta")

...

t.to_delta("/path/to/delta", mode="overwrite")
```

### Trino

The `.sql` API is now supported in Trino, enabling you to chain Ibis and SQL together.

### SQLite

Scalar Python UDFs are now supported in SQLite.

Additionally, URL parsing has been added:

In [13]:
con = ibis.sqlite.connect()

con.execute(url.host())

'ibis-project.org'

In [14]:
con.execute(url.path())

'/concepts/why_ibis'

### pandas

URL parsing support was added.

In [15]:
con = ibis.pandas.connect()

con.execute(url.host())

'ibis-project.org'

In [16]:
con.execute(url.path())

'/concepts/why_ibis'

## Functionality

### `.nunique()` supported on tables

You can now call `.nunique()` on tables to get the number of unique rows.

In [17]:
# how many unique rows are there? equivalent to `.count()` in this case
t.nunique()

344

In [18]:
# how many unique species/island/year combinations are there?
t.select("species", "island", "year").nunique()

15

### `to_sql` returns a `str` type

The `ibis.expr.sql.SQLString` type resulting from `to_sql` is now a proper `str` subclass, enabling use without casting to `str` first.

In [19]:
type(ibis.to_sql(t))

ibis.expr.sql.SQLString

In [20]:
issubclass(type(ibis.to_sql(t)), str)

True

### Allow mixing literals and columns in `ibis.array`

Note that arrays must still be of a single type.

In [21]:
ibis.array([t["species"], "hello"])

┏━━━━━━━━━━━━━━━━━━━━━┓
┃ ArrayColumn()       ┃
┡━━━━━━━━━━━━━━━━━━━━━┩
│ array<string>       │
├─────────────────────┤
│ ['Adelie', 'hello'] │
│ ['Adelie', 'hello'] │
│ ['Adelie', 'hello'] │
│ ['Adelie', 'hello'] │
│ ['Adelie', 'hello'] │
│ ['Adelie', 'hello'] │
│ ['Adelie', 'hello'] │
│ ['Adelie', 'hello'] │
│ ['Adelie', 'hello'] │
│ ['Adelie', 'hello'] │
│ …                   │
└─────────────────────┘

In [22]:
ibis.array([t["flipper_length_mm"], 42])

┏━━━━━━━━━━━━━━━┓
┃ ArrayColumn() ┃
┡━━━━━━━━━━━━━━━┩
│ array<int64>  │
├───────────────┤
│ [181, 42]     │
│ [186, 42]     │
│ [195, 42]     │
│ [None, 42]    │
│ [193, 42]     │
│ [190, 42]     │
│ [181, 42]     │
│ [195, 42]     │
│ [193, 42]     │
│ [190, 42]     │
│ …             │
└───────────────┘

### Array `concat` and `repeat` methods

You can still use `+` or `*` in typical Python fashion, with new and more explicit `concat` and `repeat` methods added in this release.

In [23]:
a = ibis.array([1, 2, 3])
b = ibis.array([4, 5])

c = a.concat(b)
c

[1, 2, 3, 4, 5]

In [24]:
c = a + b
c

[1, 2, 3, 4, 5]

In [25]:
b.repeat(2)

[4, 5, 4, 5]

In [26]:
b * 2

[4, 5, 4, 5]

### Support boolean literals in the join API

This allows for joins with boolean predicates.

In [27]:
t.join(t, True)

┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ species ┃ island    ┃ bill_length_mm ┃ bill_depth_mm ┃ flipper_length_mm ┃ body_mass_g ┃ sex    ┃ year   ┃ species_right ┃ island_right ┃ bill_length_mm_right ┃ bill_depth_mm_right ┃ flipper_length_mm_right ┃ body_mass_g_right ┃ sex_right ┃ year_right ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string  │ string    │ float64        │ float64       │ int64             │ int64       │ string │ string │ string        │ string       │ float64              │ float64             │ int64                   │ int64             │ string    │ string     │
├─────────┼───────────┼────────────────┼───────────────┼───────────────────┼─────────────┼────────┼────────┼───────────────┼──────────────┼──────────────────────┼─────────────────────┼─────────────────────────┼───────────────────┼───────────┼────────────┤
│ Adelie  │ Torgersen │           39.1 │          18.7 │               181 │        3750 │ male   │ 2007   │ Adelie        │ Torgersen    │                 39.1 │                18.7 │                     181 │              3750 │ male      │ 2007       │
│ Adelie  │ Torgersen │           39.1 │          18.7 │               181 │        3750 │ male   │ 2007   │ Adelie        │ Torgersen    │                 39.5 │                17.4 │                     186 │              3800 │ female    │ 2007       │
│ Adelie  │ Torgersen │           39.1 │          18.7 │               181 │        3750 │ male   │ 2007   │ Adelie        │ Torgersen    │                 40.3 │                18.0 │                     195 │              3250 │ female    │ 2007       │
│ Adelie  │ Torgersen │           39.1 │          18.7 │               181 │        3750 │ male   │ 2007   │ Adelie        │ Torgersen    │                  nan │                 nan │                    NULL │              NULL │ NULL      │ 2007       │
│ Adelie  │ Torgersen │           39.1 │          18.7 │               181 │        3750 │ male   │ 2007   │ Adelie        │ Torgersen    │                 36.7 │                19.3 │                     193 │              3450 │ female    │ 2007       │
│ Adelie  │ Torgersen │           39.1 │          18.7 │               181 │        3750 │ male   │ 2007   │ Adelie        │ Torgersen    │                 39.3 │                20.6 │                     190 │              3650 │ male      │ 2007       │
│ Adelie  │ Torgersen │           39.1 │          18.7 │               181 │        3750 │ male   │ 2007   │ Adelie        │ Torgersen    │                 38.9 │                17.8 │                     181 │              3625 │ female    │ 2007       │
│ Adelie  │ Torgersen │           39.1 │          18.7 │               181 │        3750 │ male   │ 2007   │ Adelie        │ Torgersen    │                 39.2 │                19.6 │                     195 │              4675 │ male      │ 2007       │
│ Adelie  │ Torgersen │           39.1 │          18.7 │               181 │        3750 │ male   │ 2007   │ Adelie        │ Torgersen    │                 34.1 │                18.1 │                     193 │              3475 │ NULL      │ 2007       │
│ Adelie  │ Torgersen │           39.1 │          18.7 │               181 │        3750 │ male   │ 2007   │ Adelie        │ Torgersen    │                 42.0 │                20.2 │                     190 │              4250 │ NULL      │ 2007       │
│ …       │ …         │              … │             … │                 … │           … │ …      │ …      │ …             │ …            │                    …

In [28]:
t.join(t, False)

┏━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ species ┃ island ┃ bill_length_mm ┃ bill_depth_mm ┃ flipper_length_mm ┃ body_mass_g ┃ sex    ┃ year   ┃ species_right ┃ island_right ┃ bill_length_mm_right ┃ bill_depth_mm_right ┃ flipper_length_mm_right ┃ body_mass_g_right ┃ sex_right ┃ year_right ┃
┡━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string  │ string │ float64        │ float64       │ int64             │ int64       │ string │ string │ string        │ string       │ float64              │ float64             │ int64                   │ int64             │ string    │ string     │
└─────────┴────────┴────────────────┴───────────────┴───────────────────┴─────────────┴────────┴────────┴───────────────┴──────────────┴──────────────────────┴─────────────────────┴─────────────────────────┴───────────────────┴───────────┴────────────┘

In [29]:
t.join(t, False, how="outer")

┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ species ┃ island    ┃ bill_length_mm ┃ bill_depth_mm ┃ flipper_length_mm ┃ body_mass_g ┃ sex    ┃ year   ┃ species_right ┃ island_right ┃ bill_length_mm_right ┃ bill_depth_mm_right ┃ flipper_length_mm_right ┃ body_mass_g_right ┃ sex_right ┃ year_right ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string  │ string    │ float64        │ float64       │ int64             │ int64       │ string │ string │ string        │ string       │ float64              │ float64             │ int64                   │ int64             │ string    │ string     │
├─────────┼───────────┼────────────────┼───────────────┼───────────────────┼─────────────┼────────┼────────┼───────────────┼──────────────┼──────────────────────┼─────────────────────┼─────────────────────────┼───────────────────┼───────────┼────────────┤
│ Adelie  │ Torgersen │           39.1 │          18.7 │               181 │        3750 │ male   │ 2007   │ NULL          │ NULL         │                  nan │                 nan │                    NULL │              NULL │ NULL      │ NULL       │
│ Adelie  │ Torgersen │           39.5 │          17.4 │               186 │        3800 │ female │ 2007   │ NULL          │ NULL         │                  nan │                 nan │                    NULL │              NULL │ NULL      │ NULL       │
│ Adelie  │ Torgersen │           40.3 │          18.0 │               195 │        3250 │ female │ 2007   │ NULL          │ NULL         │                  nan │                 nan │                    NULL │              NULL │ NULL      │ NULL       │
│ Adelie  │ Torgersen │            nan │           nan │              NULL │        NULL │ NULL   │ 2007   │ NULL          │ NULL         │                  nan │                 nan │                    NULL │              NULL │ NULL      │ NULL       │
│ Adelie  │ Torgersen │           36.7 │          19.3 │               193 │        3450 │ female │ 2007   │ NULL          │ NULL         │                  nan │                 nan │                    NULL │              NULL │ NULL      │ NULL       │
│ Adelie  │ Torgersen │           39.3 │          20.6 │               190 │        3650 │ male   │ 2007   │ NULL          │ NULL         │                  nan │                 nan │                    NULL │              NULL │ NULL      │ NULL       │
│ Adelie  │ Torgersen │           38.9 │          17.8 │               181 │        3625 │ female │ 2007   │ NULL          │ NULL         │                  nan │                 nan │                    NULL │              NULL │ NULL      │ NULL       │
│ Adelie  │ Torgersen │           39.2 │          19.6 │               195 │        4675 │ male   │ 2007   │ NULL          │ NULL         │                  nan │                 nan │                    NULL │              NULL │ NULL      │ NULL       │
│ Adelie  │ Torgersen │           34.1 │          18.1 │               193 │        3475 │ NULL   │ 2007   │ NULL          │ NULL         │                  nan │                 nan │                    NULL │              NULL │ NULL      │ NULL       │
│ Adelie  │ Torgersen │           42.0 │          20.2 │               190 │        4250 │ NULL   │ 2007   │ NULL          │ NULL         │                  nan │                 nan │                    NULL │              NULL │ NULL      │ NULL       │
│ …       │ …         │              … │             … │                 … │           … │ …      │ …      │ …             │ …            │                    …

## Refactors

Several internal refactors that shouldn't affect normal usage were made. See [the release notes](/release_notes/) for more details.

## Wrapping up

Ibis v6.1.0 brings exciting enhancements to the library that enable broader ecosystem adoption of Python standards.

As always, try Ibis by [installing](https://ibis-project.org/install/) and [getting started](/tutorial/getting_started/).

If you run into any issues or find support is lacking for your backend, [open an issue](https://github.com/ibis-project/issues/new/choose) or [discussion](https://github.com/ibis-project/discussions/new/choose) and let us know!